<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/youtube/gan/colab_gan_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Feb 21 11:19:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Set Up New Environment

You will likely need to train for >24 hours.  Colab will disconnect you.  You must be prepared to restart training when this eventually happens.  Training is divided into ticks, every so many ticks (50 by default) your neural network is evaluated and a snapshot is saved.  When CoLab shuts down, all training after the last snapshot is lost. It might seem desirable to snapshot after each tick; however, this snapshotting process itself takes nearly an hour.  It is important to learn an optimal snapshot size for your resolution and training data.

We will mount GDRIVE so that your snapshots are saved there.  You must also place your training images in GDRIVE.

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [5]:
!python --version

Python 3.7.12


You must also install NVIDIA StyleGAN2 ADA PyTorch.  We also need to downgrade PyTorch to a version that supports StyleGAN.

In [3]:
!pip install numpy==1.16.4
!pip install tensorflow-gpu==1.14
#!pip install conda
#!conda install tensorflow-gpu==1.14
!git clone https://github.com/NVlabs/stylegan2-ada
!pip install ninja

     |████████████████████████████████| 17.3 MB 4.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
xarray 0.18.2 requires numpy>=1.17, but you have numpy 1.16.4 which is incompatible.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.16.4 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.16.4 which is incompatible.
scikit-image 0.18.3 requires numpy>=1.16.5, but you have numpy 1.16.4 which is incompatible.
pywavelets 1.2.0 requires numpy>=1.17.3, but you have numpy 1.16.4 which is incompatible.
pyerfa 2.0.0.1 requires numpy>=1.17, but you have numpy 1.16.4 which is incompati

     |████████████████████████████████| 377.1 MB 8.9 kB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
     |████████████████████████████████| 488 kB 73.8 MB/s 
     |████████████████████████████████| 3.1 MB 51.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.16.4 which is incompatible.
tensorflow 2.8.0 requires tensorb

# Find Your Files

The drive is mounted to the following location.

```
/content/drive/MyDrive/data
```

It might be helpful to use an ```ls``` command to establish the exact path for your images.

In [ ]:
#!ls /content/drive/MyDrive/data/gan/images

# Convert Your Images

In [4]:
%tensorflow_version 1.x
import tensorflow as tf
#import os
#os.getcwd()
#sess = tf.Session()

TensorFlow 1.x selected.


In [20]:
!python /content/stylegan2-ada/dataset_tool.py create_from_images /content/drive/MyDrive/data/gan/dataset /content/drive/MyDrive/data/gan/images

Loading images from "/content/drive/MyDrive/data/gan/images"
Creating dataset "/content/drive/MyDrive/data/gan/dataset"
Added 2280 images.


In [22]:
!python /content/stylegan2-ada/dataset_tool.py display /content/drive/MyDrive/data/gan/dataset

Loading dataset "/content/drive/MyDrive/data/gan/dataset"
Displaying images
: cannot connect to X server 


The following command can be used to clear out the newly created dataset.  If something goes wrong and you need to clean up your images and rerun the above command, you should delete your partially created dataset directory.

In [19]:
!rm -R /content/drive/MyDrive/data/gan/dataset/*

# Clean Up your Images

It is important that all images have the same dimensions and color depth.  This code can identify images that have issues.

In [15]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/data/gan/images'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz


  0%|          | 0/2281 [00:00<?, ?it/s]

Inconsistant color format: /content/drive/MyDrive/data/gan/images/Screenshot_20210627-215429.png


# Perform Initial Training

In [5]:
!nvcc /content/stylegan2-ada/test_nvcc.cu -o /content/stylegan2-ada/test_nvcc -run

CPU says hello.
GPU says hello.


In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 25
LOSS = 'wgangp'

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA} --cmethod = {LOSS}"
!{cmd}


Training options:
{
  "G_args": {
    "func_name": "training.networks.G_main",
    "fmap_base": 8192,
    "fmap_max": 512,
    "mapping_layers": 2,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "D_args": {
    "func_name": "training.networks.D_main",
    "mbstd_group_size": 4,
    "fmap_base": 8192,
    "fmap_max": 512,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "G_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "D_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "loss_args": {
    "func_name": "training.loss.stylegan2",
    "r1_gamma": 0.8192
  },
  "augment_args": {
    "class_name": "training.augment.AdaptiveAugment",
    "tune_heuristic": "rt",
    "tune_target": 0.6,
    "apply_func": "training.augment.augment_pipeline",
    "apply_args": {
      "xflip": 1,
      "rotate90": 1,
      "xint": 1,
      "scale": 1,
      "rotate": 1,
      "aniso": 1,
      "xfrac": 1,
      "brightness": 1,
   

In [7]:
os.getcwd()

'/content'

In [ ]:
!/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap 5 --resume /content/drive/MyDrive/data/gan/experiments/00000-dataset-auto1/network-snapshot-000020.pkl --outdir /content/drive/MyDrive/data/gan/experiments --data /content/drive/MyDrive/data/gan/dataset

Usage: train.py [OPTIONS]
Try 'train.py --help' for help.

Error: --data: Path must point to a directory or zip


# Resume Training

In [27]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000028.pkl"
RESUME = os.path.join(EXPERIMENTS, "00003-dataset-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 7
LOSS = 'wgangp'

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA} --cmethod {LOSS} --aug=noaug --resume {RESUME}"
!{cmd}

Traceback (most recent call last):
  File "/content/stylegan2-ada/train.py", line 16, in <module>
    import tensorflow as tf
  File "/tensorflow-1.15.2/python3.7/tensorflow/__init__.py", line 99, in <module>
    from tensorflow_core import *
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/__init__.py", line 28, in <module>
    from tensorflow.python import pywrap_tensorflow  # pylint: disable=unused-import
  File "<frozen importlib._bootstrap>", line 1019, in _handle_fromlist
  File "/tensorflow-1.15.2/python3.7/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/tensorflow-1.15.2/python3.7/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/lib/python3.7/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/__init__.py", line 73, in <module>
    from tensorflow.python.ops.

# Resume with stylegan loss

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000028.pkl"
RESUME = os.path.join(EXPERIMENTS, "00003-dataset-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 7
LOSS = 'wgangp'

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA} --resume {RESUME}"
!{cmd}


Training options:
{
  "G_args": {
    "func_name": "training.networks.G_main",
    "fmap_base": 8192,
    "fmap_max": 512,
    "mapping_layers": 2,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "D_args": {
    "func_name": "training.networks.D_main",
    "mbstd_group_size": 4,
    "fmap_base": 8192,
    "fmap_max": 512,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "G_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "D_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "loss_args": {
    "func_name": "training.loss.stylegan2",
    "r1_gamma": 0.8192
  },
  "augment_args": {
    "class_name": "training.augment.AdaptiveAugment",
    "tune_heuristic": "rt",
    "tune_target": 0.6,
    "apply_func": "training.augment.augment_pipeline",
    "apply_args": {
      "xflip": 1,
      "rotate90": 1,
      "xint": 1,
      "scale": 1,
      "rotate": 1,
      "aniso": 1,
      "xfrac": 1,
      "brightness": 1,
   